# Build FakeHuayi backend

## Basic Use

Required files:

- ./fake_huayi/
    - \_\_init\_\_.py
    - fake_huayi.py
    - props_huayi.json
    - conf_huayi.json
    - defs_huayi.json (if Pulse Backend is applied, TODO)
- ./
    - qubits_data.csv
    - gates_data.csv

Import FakeHuayi backend

```
from fake_huayi import *
FackHuayi()    # for V1 backend
FackHuayiV2()  # for V2 backend
```

Generate noise model
```
from qiskit_aer.noise.noise_model import NoiseModel
noise_Huayi = NoiseModel.from_backend(FakeHuayi())
```

Create .json files that required to build the backend

```
from fake_huayi.HuayiBacken_build import create
c = create(backend_name="huayi",
           backend_version="x.x.x",
           qubits_data="qubits_data.csv",
           gates_data="gates_data.csv")
```

## File Structure

#### Expriment data

**qubits_data.csv** contains the information of qubits, including
- T1 time (ms)
- T2 time (ms)
- frequency (MHz)
- readout error rate
- Probability of finding 0 when prepared in 1
- Probability of finding 1 when prepared in 0
- readout length (us)

All information should be accompanied with the measurement date and time.

**gates_data.csv** contains the information of gates, including
- qubits
- gate type
- error rate
- length
- gate name (optional)

The gate error and length are measured from experiment, and should be accompanied with the measurement date and time.

#### Dictionaries of the backend properties and configurations

**props_huayi.json**
```
{'backend_name': 'fakehuayi',
 'backend_version='0.0.1',
 'last_update_date': now_time(),
 'qubits': [q1, q2, ...],
 'gates': [g1, g2, ...],
 'general': []}
```
 qubit info (q):
```
 [{'date': ['T1_date'], 'name': 'T1', 'unit': 'ms', 'value': ['T1']},
  {'date': ['T2_date'], 'name': 'T2', 'unit': 'ms', 'value': ['T2']},
  {'date': ['frequency_date'], 'name': 'frequency', 'unit': 'MHz', 'value': ['frequency']},
  {'date': ['readout_error_date'], 'name': 'readout_error', 'unit': '', 'value': ['readout_error']},
  {'date': ['prob_meas0_prep1_date'], 'name': 'prob_meas0_prep1', 'unit': '', 'value': ['prob_meas0_prep1']},
  {'date': ['prob_meas1_prep0_date'], 'name': 'prob_meas1_prep0', 'unit': '', 'value': ['prob_meas1_prep0']},
  {'date': ['readout_length_date'], 'name': 'readout_length', 'unit': 'us', 'value': ['readout_length']}]
```
gate info (g):
```
{'qubits': ['qubits'],
 'gate': ['gate'],
 'parameters': [{'date': ['error_date'],
                 'name': 'gate_error',
                 'unit': '',
                 'value': ['gate_error']},
                {'date': ['length_date'],
                 'name': 'gate_length',
                 'unit': 'ms',
                 'value': ['gate_length']}],
 'name': ['name']}
```

**conf_huayi.json**

#### Dictionary of the Noise Model

The keys in NoiseModel are
- basis gates ({'id', 'x', 'sx', 'rz', 'cx'})
- noise instructions ({'id', 'x', 'sx', 'cx', 'measure', 'reset', ''})
- noise qubits ({0, ... , 26})
- default quantum errors (set None by default)
- default readout errors (set None by default)
- local quantum errors
- local readout errors
- custom noise passes (set None by default)

In [2]:
# Generate props_huayi.json from .csv experimental data
from fake_huayi.HuayiBackend_build import create

c = create(backend_name='huayi',
       backend_version='0.0.1',
       qubits_data='qubits_data.csv', 
       gates_data='gates_data.csv')


Successfully created props_huayi.json
Successfully created conf_huayi.json


In [91]:
from fake_huayi import FakeHuayi, FakeHuayiV2
from qiskit_aer.noise.noise_model import NoiseModel

print(FakeHuayi())
print(FakeHuayiV2())

noise_Huayi = NoiseModel.from_backend(FakeHuayi())

print(noise_Huayi.__dict__)

fake_huayi
{'_basis_gates': {'x', 'cz', 'reset', 'xy', 'rx', 'ry', 'rz', 'id', 'z'}, '_noise_instructions': {'x', 'measure', 'cz', 'id', 'z'}, '_noise_qubits': {0, 1, 2, 3, 4, 5, 6, 7}, '_default_quantum_errors': {}, '_local_quantum_errors': {'id': {(0,): QuantumError([(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f2848dfdc50>, 0.9983580481106573), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f2848dfff90>, 8.48239690572117e-05), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f2848ecde90>, 3.2245211237155924e-06), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f2848e80090>, 0.0005179221224597729), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f2848e80e90>, 4.400446330123015e-08), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f2848e82d10>, 1.6727974772894745e-09), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f2848e83190>, 0.0005179221224597729), (<qiskit.circuit.quantumcircuit.QuantumCircuit

## Check the performance of the new backend

In [4]:
# generate a short circuit from ICCAD problem

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver, Psi4Driver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,BravyiKitaevMapper,QubitMapper

import Qubit_taper as Q
from qiskit.opflow import PauliSumOp


mol_geometry = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""

driver = PySCFDriver(
    atom=mol_geometry.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()
hamiltonian = qmolecule.hamiltonian
second_q_op = hamiltonian.second_q_op()
qubit_op_jw = JordanWignerMapper().map(second_q_op)
ham_jw = PauliSumOp(qubit_op_jw)

reps = 3
electron = (5,4)
Xtapered_ham, hartree = Q.taper_X_remove(ham_jw, reps, electron)
    
circ = Q.EF_ansatz(Xtapered_ham,
                   hartree,
                   Fraction=1/10,
                   reverse=True)

/home/zhouyijia/anaconda3/envs/vqe/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
/tmp/ipykernel_3434682/3846223499.py:27: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  ham_jw = PauliSumOp(qubit_op_jw)


In [101]:
from qiskit_ionq import IonQProvider
provider = IonQProvider()
fake_ionq = provider.get_backend("ionq_simulator")

from fake_huayi import FakeHuayi
from qiskit.providers.fake_provider import *

from qiskit import transpile

circ_IonQ = transpile(circ, fake_ionq, optimization_level=3)
circ_Huayi = transpile(circ, FakeHuayi(), optimization_level=3)
circ_Montreal = transpile(circ, FakeMontreal(), optimization_level=3)

print("Circuit of EF ansatz")
print(circ.draw(fold=-1))
print("Circuit of IonQ backend")
print(circ_IonQ.draw(fold=-1, idle_wires=False))
print("Circuit of Huayi backend")
print(circ_Huayi.draw(fold=-1, idle_wires=False))
print("Circuit of Montreal backend")
print(circ_Montreal.draw(fold=-1, idle_wires=False))

Circuit of EF ansatz
     ┌───┐┌───────────────┐
q_0: ┤ X ├┤1              ├
     ├───┤│               │
q_1: ┤ X ├┤               ├
     ├───┤│  Givens(T[0]) │
q_2: ┤ X ├┤               ├
     └───┘│               │
q_3: ─────┤0              ├
     ┌───┐├───────────────┤
q_4: ┤ X ├┤0              ├
     └───┘│  Givens(T[1]) │
q_5: ─────┤1              ├
          └───────────────┘
Circuit of IonQ backend
        ┌───┐    ┌───┐┌────────────┐┌───┐     
q_0: ───┤ X ├────┤ X ├┤ Ry(T[0]/2) ├┤ X ├─────
        ├───┤    └─┬─┘└────────────┘└─┬─┘     
q_1: ───┤ X ├──────┼──────────────────┼───────
        ├───┤      │                  │       
q_2: ───┤ X ├──────┼──────────────────┼───────
        ├───┤      │  ┌────────────┐  │  ┌───┐
q_3: ───┤ H ├──────■──┤ Ry(T[0]/2) ├──■──┤ H ├
     ┌──┴───┴───┐     ├────────────┤     ├───┤
q_4: ┤ Ry(-π/2) ├──■──┤ Ry(T[1]/2) ├──■──┤ H ├
     └──────────┘┌─┴─┐├────────────┤┌─┴─┐└───┘
q_5: ────────────┤ X ├┤ Ry(T[1]/2) ├┤ X ├─────
                 └───┘└────

I have manually forced the CZ gate error very large (~0.8) except for the couplings between [1,5] and [3,7]
The transpiled circuit indeed only connects the low-error qubits

The IonQ backend looks useless, because it contains all kinds of the gates, and fails to show the trapped-ion specific gates.

In [3]:
import random
from fake_huayi.HuayiBackend_build import now_time

for i in range(8):
    print(abs(random.gauss(50e3, 1e3)), now_time(),
          abs(random.gauss(1000, 50)), now_time(),
          abs(random.gauss(1, 0.05)), now_time(),
          abs(random.gauss(0, 0.05e-2)), now_time(),
          abs(random.gauss(0, 0.03e-2)), now_time(),
          abs(random.gauss(0, 0.03e-2)), now_time(),
          abs(random.gauss(0.1, 0.01)), now_time(),
          )


50442.63440391584 2023-12-11T16:40+08:00 1090.7511284308946 2023-12-11T16:40+08:00 0.9714445139978736 2023-12-11T16:40+08:00 0.0010816513440845038 2023-12-11T16:40+08:00 0.00044499168762206643 2023-12-11T16:40+08:00 0.00012638710478456226 2023-12-11T16:40+08:00 0.0960291586271935 2023-12-11T16:40+08:00
47577.886166381504 2023-12-11T16:40+08:00 1006.456658132491 2023-12-11T16:40+08:00 0.9901367341006492 2023-12-11T16:40+08:00 0.0004333787565713558 2023-12-11T16:40+08:00 0.0004782809578934633 2023-12-11T16:40+08:00 0.00012827630578504906 2023-12-11T16:40+08:00 0.08196591023660968 2023-12-11T16:40+08:00
51724.80089122017 2023-12-11T16:40+08:00 1009.6281924316586 2023-12-11T16:40+08:00 0.9812349231777265 2023-12-11T16:40+08:00 0.0007868218163987981 2023-12-11T16:40+08:00 3.468874851671884e-05 2023-12-11T16:40+08:00 0.0002864748536332695 2023-12-11T16:40+08:00 0.09648394826853678 2023-12-11T16:40+08:00
50187.59984286126 2023-12-11T16:40+08:00 987.699983148557 2023-12-11T16:40+08:00 0.944493

In [5]:
for i in range(8):
    for j in list(range(i))+list(range(i+1,8)):
        print('['+str(i)+','+str(j)+']',
            'cz',
            0.8+abs(random.gauss(0, 0.001)), now_time(),
            abs(random.gauss(150, 10)), now_time(),
            'cz'+str(i)+'_'+str(j)
            )

[0,1] cz 0.8004870142219752 2023-12-11T16:40+08:00 169.5049376078955 2023-12-11T16:40+08:00 cz0_1
[0,2] cz 0.8006877553361381 2023-12-11T16:40+08:00 142.35449635458605 2023-12-11T16:40+08:00 cz0_2
[0,3] cz 0.8013323564728543 2023-12-11T16:40+08:00 138.80950157419204 2023-12-11T16:40+08:00 cz0_3
[0,4] cz 0.801149522278485 2023-12-11T16:40+08:00 146.65502050486947 2023-12-11T16:40+08:00 cz0_4
[0,5] cz 0.8000869733918877 2023-12-11T16:40+08:00 158.47567242185195 2023-12-11T16:40+08:00 cz0_5
[0,6] cz 0.8015268411005261 2023-12-11T16:40+08:00 146.2812296812915 2023-12-11T16:40+08:00 cz0_6
[0,7] cz 0.8012935722197206 2023-12-11T16:40+08:00 155.5621333418281 2023-12-11T16:40+08:00 cz0_7
[1,0] cz 0.8008896699336079 2023-12-11T16:40+08:00 159.52780840399961 2023-12-11T16:40+08:00 cz1_0
[1,2] cz 0.8003234519034771 2023-12-11T16:40+08:00 127.48001611398368 2023-12-11T16:40+08:00 cz1_2
[1,3] cz 0.8002738716623479 2023-12-11T16:40+08:00 165.38811644955882 2023-12-11T16:40+08:00 cz1_3
[1,4] cz 0.801

In [11]:
for j in range(8):
    print('['+str(j)+']',
        'id',
        abs(random.gauss(0, 0.001)), now_time(),
        random.gauss(150, 10), now_time(),
        'id'+str(j)
        )

[0] id 0.0010940671924210368 2023-12-11T16:25+08:00 168.27651776279805 2023-12-11T16:25+08:00 id0
[1] id 0.0003104383670198981 2023-12-11T16:25+08:00 166.56220913583502 2023-12-11T16:25+08:00 id1
[2] id 0.0021836459690331976 2023-12-11T16:25+08:00 156.74012601015087 2023-12-11T16:25+08:00 id2
[3] id 0.0016762239819106415 2023-12-11T16:25+08:00 177.64232592003168 2023-12-11T16:25+08:00 id3
[4] id 0.0014258970775151481 2023-12-11T16:25+08:00 149.35882732222362 2023-12-11T16:25+08:00 id4
[5] id 0.0019301111020374707 2023-12-11T16:25+08:00 168.37745112109832 2023-12-11T16:25+08:00 id5
[6] id 0.0005204572627451805 2023-12-11T16:25+08:00 157.14406525911727 2023-12-11T16:25+08:00 id6
[7] id 0.0010374052329233591 2023-12-11T16:25+08:00 153.38160114067517 2023-12-11T16:25+08:00 id7


In [7]:
import pandas

pandas.read_csv('qubits_data.csv')

,T1,T1_date,T2,T2_date,frequency,frequency_date,readout_error,readout_error_date,prob_meas0_prep1,prob_meas0_prep1_date,prob_meas1_prep0,prob_meas1_prep0_date,readout_length,readout_length_date
0,52105.33738,2023-12-09T10:51+08:00,971.817532,2023-12-09T10:51+08:00,0.935122,2023-12-09T10:51+08:00,0.000618,2023-12-09T10:51+08:00,0.000012,2023-12-09T10:51+08:00,0.000424,2023-12-09T10:51+08:00,0.100699,2023-12-09T10:51+08:00
1,49240.82938,2023-12-09T10:51+08:00,942.204255,2023-12-09T10:51+08:00,1.011607,2023-12-09T10:51+08:00,0.000399,2023-12-09T10:51+08:00,0.000284,2023-12-09T10:51+08:00,0.000117,2023-12-09T10:51+08:00,0.089423,2023-12-09T10:51+08:00
2,50459.09178,2023-12-09T10:51+08:00,933.875260,2023-12-09T10:51+08:00,1.019855,2023-12-09T10:51+08:00,0.000133,2023-12-09T10:51+08:00,0.000134,2023-12-09T10:51+08:00,0.000121,2023-12-09T10:51+08:00,0.134335,2023-12-09T10:51+08:00
3,50262.81399,2023-12-09T10:51+08:00,948.760790,2023-12-09T10:51+08:00,0.977639,2023-12-09T10:51+08:00,0.000616,2023-12-09T10:51+08:00,0.000197,2023-12-09T10:51+08:00,0.000109,2023-12-09T10:51+08:00,0.119865,2023-12-09T10:51+08:00
4,49246.76211,2023-12-09T10:51+08:00,981.791771,2023-12-09T10:51+08:00,1.009949,2023-12-09T10:51+08:00,0.000336,2023-12-09T10:51+08:00,0.000005,2023-12-09T10:51+08:00,0.000096,2023-12-09T10:51+08:00,0.103291,2023-12-09T10:51+08:00
5,50206.23176,2023-12-09T10:51+08:00,1034.525822,2023-12-09T10:51+08:00,0.993678,2023-12-09T10:51+08:00,0.000475,2023-12-09T10:51+08:00,0.000010,2023-12-09T10:51+08:00,0.000189,2023-12-09T10:51+08:00,0.081938,2023-12-09T10:51+08:00
6,50834.08581,2023-12-09T10:51+08:00,936.641372,2023-12-09T10:51+08:00,0.931331,2023-12-09T10:51+08:00,0.000214,2023-12-09T10:51+08:00,0.000367,2023-12-09T10:51+08:00,0.000003,2023-12-09T10:51+08:00,0.111399,2023-12-09T10:51+08:00
7,50197.89568,2023-12-09T10:51+08:00,1088.049384,2023-12-09T10:51+08:00,0.937623,2023-12-09T10:51+08:00,0.000338,2023-12-09T10:51+08:00,0.000205,2023-12-09T10:51+08:00,0.000046,2023-12-09T10:51+08:00,0.101157,2023-12-09T10:51+08:00


In [8]:
pandas.read_csv('gates_data.csv')

,qubits,gate,gate_error,error_date,gate_length,length_date,name
0,[0],id,0.001094,2023-12-11T16:25+08:00,168.276518,2023-12-11T16:25+08:00,id0
1,[1],id,0.000310,2023-12-11T16:25+08:00,166.562209,2023-12-11T16:25+08:00,id1
2,[2],id,0.002184,2023-12-11T16:25+08:00,156.740126,2023-12-11T16:25+08:00,id2
3,[3],id,0.001676,2023-12-11T16:25+08:00,177.642326,2023-12-11T16:25+08:00,id3
4,[4],id,0.001426,2023-12-11T16:25+08:00,149.358827,2023-12-11T16:25+08:00,id4
...,...,...,...,...,...,...,...
75,"[7,2]",cz,0.800728,2023-12-11T16:40+08:00,130.446578,2023-12-11T16:40+08:00,cz7_2
76,"[7,3]",cz,0.000826,2023-12-11T16:40+08:00,146.528246,2023-12-11T16:40+08:00,cz7_3
77,"[7,4]",cz,0.800130,2023-12-11T16:40+08:00,145.679879,2023-12-11T16:40+08:00,cz7_4
78,"[7,5]",cz,0.800326,2023-12-11T16:40+08:00,126.877120,2023-12-11T16:40+08:00,cz7_5
